# LDA

In [4]:
import pandas as pd

ds = pd.read_parquet('./results/ds.parquet')
elapse_time = pd.read_csv('./results/elapse_time.csv')

In [5]:
from time import time

from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from utils import get_diversity, get_topics_lda
import optuna

def lda(docs):
    docs = docs.dropna()
    dictionary = Dictionary(docs)
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    def objective(trial, get_lda=False):
        num_topics = trial.suggest_int('num_topics', 5, 100)
        alpha = trial.suggest_categorical('alpha_categorical', ['symmetric', 'asymmetric', 'scalar'])
        eta = trial.suggest_categorical('eta_categorical', ['symmetric', 'auto', 'scalar'])
        if alpha == 'scalar':
            alpha = trial.suggest_float('alpha', 0.01, 1)
        if eta == 'scalar':
            eta = trial.suggest_float('eta', 0.01, 1)
        model = LdaMulticore(
            corpus=corpus,
            id2word=dictionary,
            num_topics=num_topics,
            alpha=alpha,
            eta=eta,
        )
        if get_lda:
            return model
        c = CoherenceModel(model, texts=docs, dictionary=dictionary, coherence='c_v')
        cs = c.get_coherence()
        ts = get_diversity(get_topics_lda(model, dictionary))
        return cs * ts

    t_start = time()
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=2) # FIXME: change n to 20

    best_trial = study.best_trial
    best_lda = objective(best_trial, get_lda=True)
    t_end = time()

    return {
        'model': best_lda,
        'time': t_end - t_start,
    }

d:\repo\lda-bertopic\.venv\lib\site-packages\umap\umap_.py:660: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()


In [6]:
def save_models_lda():
    r_time = []
    for v in ds:
        model = lda(ds[v])
        model['model'].save(f'./results/models_lda/{v}')
        r_time.append(model['time'])
    elapse_time['lda_training'] = pd.Series(r_time)

In [7]:
save_models_lda()

[I 2023-08-22 12:30:53,448] A new study created in memory with name: no-name-0f4fb4fe-fca7-45f0-a603-ce48df168b50
[I 2023-08-22 12:31:01,821] Trial 0 finished with value: 0.06670738096745027 and parameters: {'num_topics': 9, 'alpha_categorical': 'scalar', 'eta_categorical': 'symmetric', 'alpha': 0.9457040041108908}. Best is trial 0 with value: 0.06670738096745027.
[I 2023-08-22 12:31:10,563] Trial 1 finished with value: 0.09778219631563202 and parameters: {'num_topics': 6, 'alpha_categorical': 'scalar', 'eta_categorical': 'symmetric', 'alpha': 0.5020961367739877}. Best is trial 1 with value: 0.09778219631563202.
[I 2023-08-22 12:31:15,795] A new study created in memory with name: no-name-8f5f1d19-165c-4775-8049-0dde7ee19101
[I 2023-08-22 12:31:23,474] Trial 0 finished with value: 0.035998249244859906 and parameters: {'num_topics': 10, 'alpha_categorical': 'asymmetric', 'eta_categorical': 'scalar', 'eta': 0.7075572669291939}. Best is trial 0 with value: 0.035998249244859906.
[I 2023-08-

In [8]:
elapse_time

,variant,nlp,preprocessing,lda_training
0,dCL,26.75152,0.015996,22.314663
1,DCL,26.75152,0.015996,21.944010
2,dLWN,26.75152,0.097994,26.633091
3,DLWN,26.75152,0.184994,18.792636


# BERTopic

In [10]:
from time import time

from bertopic import BERTopic
import pandas as pd

embedding_model = "paraphrase-multilingual-MiniLM-L12-v2"

def save_models_bertopic():
    path_prefix = './results/models_bertopic/'
    r_time = []
    for v in ds:
        t_start = time()
        docs = ds[v].dropna()
        docs = [' '.join(doc) for doc in docs]
        bertopic = BERTopic(language='multilingual', embedding_model=embedding_model)
        model = bertopic.fit(docs)
        t_end = time()
        model.save(
            f"{path_prefix}{v}",
            serialization="safetensors",
            save_embedding_model=embedding_model,
            save_ctfidf=True,
        )
        r_time.append(t_end - t_start)
    elapse_time['bertopic_training'] = pd.Series(r_time)

In [11]:
save_models_bertopic()

In [12]:
elapse_time

,variant,nlp,preprocessing,lda_training,bertopic_training
0,dCL,26.75152,0.015996,22.314663,57.348722
1,DCL,26.75152,0.015996,21.944010,11.408381
2,dLWN,26.75152,0.097994,26.633091,35.589177
3,DLWN,26.75152,0.184994,18.792636,12.978124


In [13]:
elapse_time.to_csv('./results/elapse_time.csv', index=False)